In [6]:
import pandas as pd
import os

# Replace 'your_folder_path' with the path to the folder containing the Excel files
folder_path = os.getcwd()

# Initialize an empty DataFrame to store the results
df = pd.DataFrame(columns=['File', 'Sheet Count'])

# Iterate through each file in the folder
for file in os.listdir(folder_path):
    if file.endswith('.xlsx') or file.endswith('.xls'):
        file_path = os.path.join(folder_path, file)
        # Count the number of sheets in the Excel file
        xls = pd.ExcelFile(file_path)
        sheet_count = len(xls.sheet_names)
        # Append the results to the DataFrame
        df = pd.concat([df, pd.DataFrame({'File': [file], 'Sheet Count': [sheet_count]})], ignore_index=True)


# Display the DataFrame
print(df)


                                File Sheet Count
0      blackwith18-50_converted.xlsx          77
1        blackwith51-_converted.xlsx          78
2   blackwithout18-50_converted.xlsx          78
3     blackwithout51-_converted.xlsx          76
4      whitewith18-50_converted.xlsx          76
5      whitewith51-65_converted.xlsx          77
6      whitewith66-75_converted.xlsx          78
7        whitewith76-_converted.xlsx          75
8   whitewithout18-40_converted.xlsx          76
9   whitewithout41-62_converted.xlsx          78
10    whitewithout63-_converted.xlsx          78


In [7]:
import pandas as pd
import os

# Specify the path to the folder containing the Excel files
#folder_path = '/Users/ketan/Desktop/allofus/all_of_us'

# Step 1: Identify the complete set of unique tab names
all_tabs = set()
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)
        xls = pd.ExcelFile(file_path)
        all_tabs.update(xls.sheet_names)

# Step 2: Read each workbook as before
dataframes = {}
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)
        df_name = os.path.splitext(filename)[0]
        dataframes[df_name] = pd.read_excel(file_path, sheet_name=None)
print(dataframes['blackwith18-50_converted'])

# all_tabs now contains the names of all unique tabs across all workbooks
# dataframes is a dictionary with each key being a workbook name and each value being a dictionary of DataFrames


{'ACTA1':   Chromosome   Position     Alleles Allele Count  Allele Number   
0       chr1  229432340  ['C', 'T']          [7]           3144  \

  Allele Frequency  
0       [0.002226]  , 'ABCG8':     Chromosome  Position     Alleles Allele Count  Allele Number   
0         chr2  43839108  ['G', 'C']        [207]           3144  \
1         chr2  43839131  ['G', 'C']          [0]           3142   
2         chr2  43839156  ['G', 'A']         [23]           3144   
3         chr2  43839213  ['C', 'A']         [17]           3144   
4         chr2  43839216  ['G', 'A']          [1]           3144   
..         ...       ...         ...          ...            ...   
449       chr2  43851711  ['C', 'T']          [1]           3144   
450       chr2  43851714  ['G', 'A']        [282]           3144   
451       chr2  43851748  ['G', 'C']          [1]           3144   
452       chr2  43851752  ['G', 'A']         [17]           3144   
453       chr2  43851798  ['C', 'A']          [1]      

In [8]:
import pandas as pd
import ast


# Function to process DataFrames for each gene within a single population group
def process_genes_for_population(df_dict):
    population_data = []
    
    count=0
    for gene_name, df in df_dict.items():
        for index, row in df.iterrows():
            allele_count_str = row['Allele Count']
            try:
                # Converting string representation of a list to an actual list and getting the first element
                allele_count = ast.literal_eval(allele_count_str)[0] if allele_count_str else 0
            except (ValueError, SyntaxError):
                allele_count = 0  # Default to 0 in case of error
            
            # Add a row of data for this gene (if there is an array there) or make 0
            if pd.isna(row['Allele Number']):
                count = count + 1
                population_data.append({
                'Gene-Position': f"{gene_name}_{row['Position']}",
                'Allele Count': 99999999,
                'Allele Number': int(99999999)})
            else:
                population_data.append({
                'Gene-Position': f"{gene_name}_{row['Position']}",
                'Allele Count': allele_count,
                'Allele Number': int(row['Allele Number'])})

    print(count)

            
    return pd.DataFrame(population_data)

# Dictionary to hold the DataFrame for each population group
population_dataframes = {}

# Loop through all population groups in the `dataframes` dictionary
for population_name, df_dict in dataframes.items():
    print(population_name)
    population_dataframes[population_name] = process_genes_for_population(df_dict)


# At this point, `population_dataframes` contains a separate DataFrame for each population group
# You can access them like this:
blackwith18_50_df = population_dataframes['blackwith18-50_converted'] #for example
blackwith51__df = population_dataframes['blackwith51-_converted']
blackwithout18_50_df = population_dataframes['blackwithout18-50_converted'] 
blackwithout51__df = population_dataframes['blackwithout51-_converted'] 
whitewith18_50_df = population_dataframes['whitewith18-50_converted'] 
whitewith51_65_df = population_dataframes['whitewith51-65_converted'] 
whitewith66_75_df = population_dataframes['whitewith66-75_converted'] 
whitewith76__df = population_dataframes['whitewith76-_converted'] 
whitewithout18_40_df = population_dataframes['whitewithout18-40_converted'] 
whitewithout41_62_df = population_dataframes['whitewithout41-62_converted'] 
whitewithout63__df = population_dataframes['whitewithout63-_converted']



blackwith18-50_converted
58
blackwith51-_converted
49
blackwithout18-50_converted
50
blackwithout51-_converted
18
whitewith18-50_converted
44
whitewith51-65_converted
52
whitewith66-75_converted
57
whitewith76-_converted
54
whitewithout18-40_converted
56
whitewithout41-62_converted
50
whitewithout63-_converted
56


In [29]:
# Ensure all relevant DataFrames are set with 'Gene-Position' as the index
for df in population_dataframes.values():
    df.set_index('Gene-Position', inplace=True)

# prove combinations are working
# print(population_dataframes['blackwith18-50_updated'])
# print(population_dataframes['blackwith51-_converted'])

df1 = blackwith18_50_df
df2 = blackwith51__df

combined_allele_count = df1['Allele Count'].add(df2['Allele Count'], fill_value=0).astype(int)
combined_allele_number = df1['Allele Number'].add(df2['Allele Number'], fill_value=0).astype(int) # need to set allele number to smth other than 0

combined_df = pd.DataFrame({
    'Allele Count': combined_allele_count,
    'Allele Number': combined_allele_number
})
combined_df.index.name = 'Gene-Position'

specific_gene_position = 'ACTA1_229432340'
if specific_gene_position in combined_df.index:
    specific_values = combined_df.loc[specific_gene_position]
    print(f"Combined values for {specific_gene_position}:")
    print(specific_values)
else:
    print(f"{specific_gene_position} not found in the combined DataFrame.")

pd.set_option('display.max_rows', None)
#print(combined_df)



# # List of DataFrames to aggregate. Add more as needed.
# dfs_to_aggregate = [blackwith18_50_df, blackwithout18_50_df]

# # Concatenate and sum
# aggregated_df = pd.concat(dfs_to_aggregate, axis=0).groupby(level=0).sum(min_count=1).fillna(0).astype(int)

# # Resetting the index if needed, to use 'Gene-Position' as a column again
# aggregated_df.reset_index(inplace=True)

# # Display the aggregated DataFrame
# print(aggregated_df)


Combined values for ACTA1_229432340:
Allele Count        17
Allele Number    10436
Name: ACTA1_229432340, dtype: int32


In [30]:
# combine all dataframes into Black/White and CVD/no CVD
# basic combinations
blackwith1 = blackwith18_50_df
blackwith2 = blackwith51__df
blackwithout1 = blackwithout18_50_df
blackwithout2 = blackwithout51__df
whitewith1 = whitewith18_50_df
whitewith2 = whitewith51_65_df
whitewith3 = whitewith66_75_df
whitewith4 = whitewith76__df
whitewithout1 = whitewithout18_40_df
whitewithout2 = whitewithout41_62_df
whitewithout3 = whitewithout63__df

#Black with combination
black_with_combined_allele_count = blackwith1['Allele Count'].add(blackwith2['Allele Count'], fill_value=0).astype(int)
black_with_combined_allele_number = blackwith1['Allele Number'].add(blackwith2['Allele Number'], fill_value=0).astype(int)
black_with_combined_df = pd.DataFrame({
    'Allele Count': black_with_combined_allele_count,
    'Allele Number': black_with_combined_allele_number
})
black_with_combined_df.index.name = 'Gene-Position'

#Black without combination
black_without_combined_allele_count = blackwithout1['Allele Count'].add(blackwithout2['Allele Count'], fill_value=0).astype(int)
black_without_combined_allele_number = blackwithout1['Allele Number'].add(blackwithout2['Allele Number'], fill_value=0).astype(int)
black_without_combined_df = pd.DataFrame({
    'Allele Count': black_without_combined_allele_count,
    'Allele Number': black_without_combined_allele_number
})
black_without_combined_df.index.name = 'Gene-Position'

#White with combination
white_with_combined_allele_count = whitewith1['Allele Count'].add(whitewith2['Allele Count'], fill_value=0).add(whitewith3['Allele Count'], fill_value=0).add(whitewith4['Allele Count'], fill_value=0).astype(int)
white_with_combined_allele_number = whitewith1['Allele Number'].add(whitewith2['Allele Number'], fill_value=0).add(whitewith3['Allele Count'], fill_value=0).add(whitewith4['Allele Count'], fill_value=0).astype(int)
white_with_combined_df = pd.DataFrame({
    'Allele Count': white_with_combined_allele_count,
    'Allele Number': white_with_combined_allele_number
})
white_with_combined_df.index.name = 'Gene-Position'

#White without combination
white_without_combined_allele_count = whitewithout1['Allele Count'].add(whitewithout2['Allele Count'], fill_value=0).add(whitewithout3['Allele Count'], fill_value=0).astype(int)
white_without_combined_allele_number = whitewithout1['Allele Number'].add(whitewithout2['Allele Number'], fill_value=0).add(whitewithout3['Allele Count'], fill_value=0).astype(int)
white_without_combined_df = pd.DataFrame({
    'Allele Count': white_without_combined_allele_count,
    'Allele Number': white_without_combined_allele_number
})
white_without_combined_df.index.name = 'Gene-Position'


In [31]:
# Black total combination
black_total_combined_allele_count = black_with_combined_df['Allele Count'].add(black_without_combined_df['Allele Count'], fill_value=0).astype(int)
black_total_combined_allele_number = black_with_combined_df['Allele Number'].add(black_without_combined_df['Allele Number'], fill_value=0).astype(int)

black_combined_df = pd.DataFrame({
    'Allele Count': black_total_combined_allele_count,
    'Allele Number': black_total_combined_allele_number
})
black_combined_df.index.name = 'Gene-Position'

# White total combination
white_total_combined_allele_count = white_with_combined_df['Allele Count'].add(white_without_combined_df['Allele Count'], fill_value=0).astype(int)
white_total_combined_allele_number = white_with_combined_df['Allele Number'].add(white_without_combined_df['Allele Number'], fill_value=0).astype(int)

white_combined_df = pd.DataFrame({
    'Allele Count': white_total_combined_allele_count,
    'Allele Number': white_total_combined_allele_number
})
white_combined_df.index.name = 'Gene-Position'

# CVD total combination
cvd_combined_allele_count = white_with_combined_df['Allele Count'].add(black_with_combined_df['Allele Count'], fill_value=0).astype(int)
cvd_combined_allele_number = white_with_combined_df['Allele Number'].add(black_with_combined_df['Allele Number'], fill_value=0).astype(int)

cvd_combined_df = pd.DataFrame({
    'Allele Count': cvd_combined_allele_count,
    'Allele Number': cvd_combined_allele_number
})
cvd_combined_df.index.name = 'Gene-Position'

# no CVD total combination
no_cvd_combined_allele_count = white_without_combined_df['Allele Count'].add(black_without_combined_df['Allele Count'], fill_value=0).astype(int)
no_cvd_combined_allele_number = white_without_combined_df['Allele Number'].add(black_without_combined_df['Allele Number'], fill_value=0).astype(int)

no_cvd_combined_df = pd.DataFrame({
    'Allele Count': no_cvd_combined_allele_count,
    'Allele Number': no_cvd_combined_allele_number
})
no_cvd_combined_df.index.name = 'Gene-Position'

In [42]:
no_cvd_combined_df.head(5)

df_no_cvd = no_cvd_combined_df[no_cvd_combined_df['Allele Count'] < 99999999]
df_cvd = cvd_combined_df[cvd_combined_df['Allele Count'] < 99999999]
df_white = white_combined_df[white_combined_df['Allele Count'] < 99999999]
df_black = black_combined_df[black_combined_df['Allele Count'] < 99999999]

In [43]:
df_no_cvd.shape

(183345, 2)

In [ ]:
# more efficient way to do fisher's
from scipy.stats import fisher_exact
import pandas as pd
import numpy as np

# Preprocessing to ensure gene positions are directly accessible
# Convert dataframes to dictionaries for faster access
df1_dict = df_cvd[['Allele Count', 'Allele Number']].to_dict('index')
df2_dict = df_no_cvd[['Allele Count', 'Allele Number']].to_dict('index')

# Function to perform Fisher's Exact Test for a specific gene position, using dictionaries for faster lookup
def fishers_test_for_gene_position(gene_position, df1_dict, df2_dict):
    if gene_position in df1_dict and gene_position in df2_dict:
        row_df1 = df1_dict[gene_position]
        row_df2 = df2_dict[gene_position]

        contingency_table = [
            [row_df1['Allele Count'], row_df1['Allele Number'] - row_df1['Allele Count']],
            [row_df2['Allele Count'], row_df2['Allele Number'] - row_df2['Allele Count']]
        ]

        odds_ratio, p_value = fisher_exact(contingency_table, alternative='two-sided')

        return odds_ratio, p_value
    else:
        return None

# Get the common gene positions between both populations
common_gene_positions = set(df1_dict.keys()).intersection(set(df2_dict.keys()))

# Perform Fisher's Exact Test on the common gene positions
fishers_results = {gene_position: fishers_test_for_gene_position(gene_position, df1_dict, df2_dict) for gene_position in common_gene_positions if fishers_test_for_gene_position(gene_position, df1_dict, df2_dict) is not None}

# Create a dataframe from the results
valid_fishers_results_df = pd.DataFrame([
    {'Gene-Position': gene_pos, 'Odds_Ratio': res[0], 'P_Value': res[1]}
    for gene_pos, res in fishers_results.items()
], columns=['Gene-Position', 'Odds_Ratio', 'P_Value'])

# Filter out significant results based on a significance level (alpha)
alpha = 0.05
significant_results_df = valid_fishers_results_df[valid_fishers_results_df['P_Value'] < alpha]
print(significant_results_df)

# Display the significant results, sorted by P_Value
# print(significant_results_df.sort_values(by='P_Value').head())


In [ ]:
len(significant_results_df)

In [ ]:
from scipy.stats import fisher_exact
import pandas as pd
import numpy as np

# Function to perform Fisher's Exact Test for a specific gene position
def fishers_test_for_gene_position(gene_position, df1, df2):
    # Extract rows for the specific gene position
    row_df1 = df1.loc[df1.index == gene_position]
    row_df2 = df2.loc[df2.index == gene_position]

    # Check if the gene position is found in both dataframes and that no values are NaN
    if not row_df1.empty and not row_df2.empty and \
       not row_df1.isnull().values.any() and not row_df2.isnull().values.any():

        # Build the contingency table with allele counts in one population vs. the other
        contingency_table = [
            [row_df1.iloc[0]['Allele Count'], row_df1.iloc[0]['Allele Number'] - row_df1.iloc[0]['Allele Count']],
            [row_df2.iloc[0]['Allele Count'], row_df2.iloc[0]['Allele Number'] - row_df2.iloc[0]['Allele Count']]
        ]

        # Perform Fisher's Exact Test
        odds_ratio, p_value = fisher_exact(contingency_table, alternative='two-sided')

        return odds_ratio, p_value
    else:
        # Return None if data is incomplete or contains NaN
        return None

# Get the common gene positions between both populations
# common_gene_positions = set(cvd_combined_df['Gene-Position']).intersection(no_cvd_combined_df['Gene-Position'])
common_gene_positions = cvd_combined_df.index.intersection(no_cvd_combined_df.index)
print(common_gene_positions)

# Perform Fisher's Exact Test on the common gene positions
fishers_results = {}
for gene_position in common_gene_positions:              #Put one class here, and another here to compare!
    result = fishers_test_for_gene_position(gene_position, cvd_combined_df, no_cvd_combined_df)
    if result:
        fishers_results[gene_position] = result

# Create dataframes from the results, separating valid and invalid results
valid_fishers_results_df = pd.DataFrame([
    {'Gene-Position': gene_pos, 'Odds_Ratio': res[0], 'P_Value': res[1]}
    for gene_pos, res in fishers_results.items() if res is not None
], columns=['Gene-Position', 'Odds_Ratio', 'P_Value'])

# Filter out significant results
alpha = 0.05  # significance level
significant_results_df = valid_fishers_results_df[valid_fishers_results_df['P_Value'] < alpha]

# Display the significant results
print(significant_results_df.sort_values(by='P_Value').head())

# maybe hit up simin liu? who is helping us publish

# get list of genes that cause cvd (cvd vs non cvd significant difference)
# prove genetic risk score (cvd group should have statisitcally significantly higher score than non cvd)
# compare black vs white genetic risk score (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6941594/)
# 

In [ ]:
significant_results_df

: 